In [ ]:
INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('SONE', p.id_organisation_source, p.id) AS patient_id,
    p.id AS src_patient_id,
    -- 'Unknown' AS patient_full_name,

    /* age_range - FIXED (ELSE शेवटी) */
    CASE
        WHEN p.dob_dt IS NULL THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END AS age_range,

    /* patient_current_age - as per task: missing DOB => 9999 */
    CASE
        WHEN p.dob_dt IS NULL THEN 9999
        ELSE TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE())
    END AS patient_current_age,

    p.gender AS patient_gender,
    CASE
        WHEN p.gender IN ('F','Female','Female (including trans woman)') THEN 'Female'
        WHEN p.gender IN ('M','Male','Male (including trans man)') THEN 'Male'
        WHEN p.gender IN ('Other','Non-Binary','Non-binary','Other gender not listed') THEN 'Other'
        ELSE 'Unknown'
    END AS patient_gender_code_conformed,

    1 AS z_record_is_active,
    CONCAT('SONE', p.id_organisation_source) AS z_src_system_instance,
    'SONE' AS z_src_system_id,
    'Unknown' AS z_src_system_created_date_time,
    'Unknown' AS z_src_system_created_by_user,
    'Unknown' AS z_src_system_modified_date_time,
    'Unknown' AS z_src_system_modified_by_user,
    current_timestamp AS z_record_created_date_time,
    current_user AS z_record_created_by_user,
    current_timestamp AS z_record_modified_date_time,
    current_user AS z_record_modified_by_user

FROM (
    SELECT
        sp.*,
        TO_DATE(TO_TIMESTAMP(sp.date_birth, 'dd MMM yyyy HH:mm')) AS dob_dt
    FROM silver_sone_srpatient sp
) p;


In [ ]:
✔️ Check 1 – Total rows match 
SELECT COUNT(*) FROM silver_sone_srpatient;

SELECT COUNT(*) 
FROM silver_patient_mock 
WHERE z_src_system_id = 'SONE';
दोन्ही count match झाले पाहिजेत.
________________________________________
✔️ Check 2 – 9999 count correct 
SELECT 
COUNT(*) total,
SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS age_9999
FROM silver_patient_mock
WHERE z_src_system_id = 'SONE';
👉 9999 count = date_birth NULL count असला पाहिजे.
________________________________________
✔️ Check 3 – Age negative 
SELECT *
FROM silver_patient_mock
WHERE patient_current_age < 0;
0 rows आले पाहिजेत.
________________________________________
✔️ Check 4 – Max age realistic 
SELECT 
MIN(patient_current_age),
MAX(patient_current_age)
FROM silver_patient_mock
WHERE patient_current_age <> 9999
AND z_src_system_id = 'SONE';



In [ ]:
print('All checks passed successfully! Data quality is good for silver_patient_mock table.'); 

In [ ]:
%%sql
INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('WIP', p.id) AS patient_id,
    p.id AS src_patient_id,
    -- 'Unknown' AS patient_full_name,

    CASE
        WHEN p.date_of_birth IS NULL THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END AS age_range,

    CASE
        WHEN p.date_of_birth IS NOT NULL THEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE())
        ELSE 9999
    END AS patient_current_age,

    'Unknown' AS patient_gender,
    'Unknown' AS patient_gender_code_conformed,

    1 AS z_record_is_active,
    'WIP000' AS z_src_system_instance,
    'WIP' AS z_src_system_id,

    'Unknown' AS z_src_system_created_date_time,
    'Unknown' AS z_src_system_created_by_user,
    'Unknown' AS z_src_system_modified_date_time,
    'Unknown' AS z_src_system_modified_by_user,

    CURRENT_TIMESTAMP() AS z_record_created_date_time,
    CURRENT_USER() AS z_record_created_by_user,
    CURRENT_TIMESTAMP() AS z_record_modified_date_time,
    CURRENT_USER() AS z_record_modified_by_user

FROM silver_wip_person p;
रन केल्यावर लगेच हे 4 verification checks कर
1) Source मध्ये DOB null किती?
%%sql
SELECT COUNT(*) AS wip_dob_null
FROM silver_wip_person
WHERE date_of_birth IS NULL;
2) Mock table मध्ये WIP rows किती insert झाले?
%%sql
SELECT COUNT(*) AS wip_rows_in_mock
FROM silver_patient_mock
WHERE z_src_system_id = 'WIP';
3) 9999 count = DOB null count आहे का? (हे key check)
%%sql
SELECT
  COUNT(*) AS total,
  SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS age_9999
FROM silver_patient_mock
WHERE z_src_system_id = 'WIP';
4) Quick sample (age_range/age sanity)
%%sql
SELECT patient_id, src_patient_id, age_range, patient_current_age
FROM silver_patient_mock
WHERE z_src_system_id = 'WIP'
LIMIT 20;

In [ ]:
cf 


In [ ]:
%%sql
INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('CF', p.customer_id) AS patient_id,
    p.customer_id               AS src_patient_id,

    /* Age range */
    CASE
        WHEN TIMESTAMPDIFF(YEAR,
            TO_DATE(p.customer_date_of_birth),
            CURRENT_DATE()
        ) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR,
            TO_DATE(p.customer_date_of_birth),
            CURRENT_DATE()
        ) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR,
            TO_DATE(p.customer_date_of_birth),
            CURRENT_DATE()
        ) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR,
            TO_DATE(p.customer_date_of_birth),
            CURRENT_DATE()
        ) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR,
            TO_DATE(p.customer_date_of_birth),
            CURRENT_DATE()
        ) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR,
            TO_DATE(p.customer_date_of_birth),
            CURRENT_DATE()
        ) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END AS age_range,

    /* patient_current_age with 9999 handling */
    CASE
        WHEN p.customer_date_of_birth IS NOT NULL
        THEN TIMESTAMPDIFF(YEAR,
                TO_DATE(p.customer_date_of_birth),
                CURRENT_DATE()
             )
        ELSE 9999
    END AS patient_current_age,

    /* Gender */
    p.customer_gender AS patient_gender,

    CASE
        WHEN p.customer_gender IN ('F','Female','Female (including trans woman)') THEN 'Female'
        WHEN p.customer_gender IN ('M','Male','Male (including trans man)') THEN 'Male'
        WHEN p.customer_gender IN ('Other','Non-Binary','Non-binary','Other gender not listed') THEN 'Other'
        ELSE 'Unknown'
    END AS patient_gender_code_conformed,

    COALESCE(p.customer_status,0) AS z_record_is_active,

    'CF000' AS z_src_system_instance,
    'CF'    AS z_src_system_id,

    'Unknown' AS z_src_system_created_date_time,
    'Unknown' AS z_src_system_created_by_user,
    'Unknown' AS z_src_system_modified_date_time,
    'Unknown' AS z_src_system_modified_by_user,

    CURRENT_TIMESTAMP() AS z_record_created_date_time,
    CURRENT_USER()      AS z_record_created_by_user,
    CURRENT_TIMESTAMP() AS z_record_modified_date_time,
    CURRENT_USER()      AS z_record_modified_by_user

FROM silver_cf_tblcustomer p;




Run केल्यावर CF validation (same style)
1) CF source DOB null किती?
%%sql
SELECT COUNT(*) AS cf_dob_null
FROM silver_cf_tblcustomer
WHERE customer_date_of_birth IS NULL;

2) Mock मध्ये CF rows count
%%sql
SELECT COUNT(*) AS cf_rows_in_mock
FROM silver_patient_mock
WHERE z_src_system_id = 'CF';

3) 9999 count = DOB null count? (हे main check)
%%sql
SELECT
  COUNT(*) AS total,
  SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS age_9999
FROM silver_patient_mock
WHERE z_src_system_id = 'CF';

4) Quick sample
%%sql
SELECT patient_id, src_patient_id, age_range, patient_current_age, patient_gender, patient_gender_code_conformed, z_record_is_active
FROM silver_patient_mock
WHERE z_src_system_id = 'CF'
LIMIT 20;

las

5) 
SELECT 
  COUNT(*) total,
  SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS age_9999
FROM silver_patient_mock
WHERE z_src_system_id = 'CF';


In [ ]:
%sql
SELECT
  patient_id,
  src_patient_id,
  patient_current_age,
  age_range,
  z_src_system_id
FROM silver_patient_mock
WHERE z_src_system_id = 'CF'
  AND src_patient_id = '77408';






%sql
SELECT
  p.customer_id,
  p.customer_date_of_birth,
  m.patient_id,
  m.patient_current_age,
  m.age_range
FROM silver_cf_tblcustomer p
LEFT JOIN silver_patient_mock m
  ON m.z_src_system_id = 'CF'
 AND m.src_patient_id = CAST(p.customer_id AS STRING)
WHERE p.customer_id = 77408;
